In [1]:
from dask_jobqueue import SLURMCluster

# Compose SLURM script
cluster = SLURMCluster(queue='caslake', cores=10, memory='40GB', 
                       processes=10, walltime='01:00:00', interface='ib0',
                       job_extra=['--account=macs30123']
                      )

# Request resources
cluster.scale(jobs=1)

/software/python-anaconda-2022.05-el8-x86_64/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41265 instead
  warnings.warn(


In [2]:
from dask.distributed import Client

client = Client(cluster)
client

<Client: 'tcp://172.25.0.65:33829' processes=10 threads=10, memory=37.30 GiB>

In [3]:
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split, IncrementalSearchCV
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, mean_squared_error
import matplotlib.pyplot as plt
import numpy as np

# ---------- Step 1: Load Data ----------
df = dd.read_parquet("data/hmda_filtered_2018_2023.parquet")

# ---------- Step 2: Prepare features ----------
features = [
    'loan_amount',
    'income',
    'debt_to_income_ratio',
    'applicant_age',
    'derived_race',
    'derived_ethnicity',
    'loan_type'
]



In [4]:
df.head()

,activity_year,action_taken,rate_spread,total_loan_costs,loan_amount,income,debt_to_income_ratio,applicant_credit_score_type,applicant_age,derived_race,...,applicant_race_1,applicant_ethnicity_1,co_applicant_race_1,co_applicant_ethnicity_1,loan_purpose,loan_term,interest_rate,property_value,occupancy_type,lien_status
0,2018,1,1.305,7822.87,135000,67,46,1,45-54,White,...,5,2,8,5,32,240,4.875,195000,1,1
1,2018,1,0.057,6859.57,235000,84,50%-60%,9,45-54,White,...,5,1,5,2,32,360,4.375,275000,1,1
2,2018,3,None,None,185000,45,>60%,3,35-44,White,...,5,2,5,2,32,360,None,355000,1,1
3,2018,3,None,None,285000,78,42,1,35-44,White,...,5,1,8,5,31,360,None,425000,1,1
4,2018,3,None,None,205000,36,>60%,1,25-34,White,...,5,1,8,5,32,360,None,265000,1,1


In [5]:
df.columns

Index(['activity_year', 'action_taken', 'rate_spread', 'total_loan_costs',
       'loan_amount', 'income', 'debt_to_income_ratio',
       'applicant_credit_score_type', 'applicant_age', 'derived_race',
       'derived_ethnicity', 'tract_minority_population_percent',
       'tract_to_msa_income_percentage', 'derived_msa_md', 'state_code',
       'county_code', 'census_tract', 'lei', 'loan_type',
       'derived_loan_product_type', 'applicant_sex', 'applicant_race_1',
       'applicant_ethnicity_1', 'co_applicant_race_1',
       'co_applicant_ethnicity_1', 'loan_purpose', 'loan_term',
       'interest_rate', 'property_value', 'occupancy_type', 'lien_status'],
      dtype='object')